In [27]:

from dataclasses import dataclass,field
import numpy as np

from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import cmasher as cmr

import networkx as nx
from collections import Counter

import itertools

In [73]:
@dataclass
class Hail:
    pos : np.array
    vel : np.array

    @classmethod
    def fromfile(cls,row):
        p,v = row.split(" @ ")
        return cls(np.array([int(c) for c in p.split(", ")],dtype=int),np.array([int(c) for c in v.split(", ")],dtype=int))
    
    def inter2d(self,other):
        dpos = other.pos[:2]-self.pos[:2]
        perpother = np.array([-other.vel[1],other.vel[0]])
        if (denom:=np.dot(perpother,self.vel[:2]))!=0:
            t=np.dot(dpos,perpother)/denom
            return self.pos[:2]+ t*self.vel[:2]
        elif np.cross(dpos,self.vel[:2])==0:
            return self.pos[:2]
        else: return None

    def inter2dfuture(self,other):
        dpos = other.pos[:2]-self.pos[:2]
        perpself = np.array([-self.vel[1],self.vel[0]])
        perpother = np.array([-other.vel[1],other.vel[0]])
        cross = np.cross(other.vel[:2],self.vel[:2])
        if cross!=0:
            tself=np.dot(dpos,perpother)/cross
            tother=np.dot(dpos,perpself)/cross
            # print(tself,tother)
            if tself>=0 and tother >=0:
                return self.pos[:2]+ tself*self.vel[:2]
            else: return None
        elif np.cross(dpos,self.vel[:2])==0:
            if np.dot(other.vel[:2]-self.vel[:2],dpos)>0:
                return self.pos[:2]+np.norm(dpos)/np.norm(other.vel[:2]-self.vel[:2])*self.vel[:2],0,0
        else: return None

    def inter2dinarea(self,other,area):
        inter = self.inter2dfuture(other)
        if inter is not None:
            
            return np.all((area[0]<=inter)&(inter<=area[1]))
        else: return False




In [74]:
with open("input24.txt") as f:
    hails =[Hail.fromfile(row.rstrip()) for row in f]

# Part 1

In [75]:
testarea = (200_000_000_000_000,400_000_000_000_000)

In [76]:
ninter = 0
for h1,h2 in tqdm(itertools.combinations(hails,2)):
    ninter += h1.inter2dinarea(h2,testarea)

44850it [00:00, 63828.52it/s]


In [77]:
ninter

20963

In [67]:
ex = """19, 13, 30 @ -2,  1, -2
18, 19, 22 @ -1, -1, -2
20, 25, 34 @ -2, -2, -4
12, 31, 28 @ -1, -2, -1
20, 19, 15 @  1, -5, -3"""
exhail = [Hail.fromfile(row.rstrip()) for row in ex.splitlines()]

In [68]:
ninter = 0
for h1,h2 in tqdm(itertools.combinations(exhail,2)):
    print(h1.inter2d(h2))
    ninter += h1.inter2dinarea(h2,(7,27))
ninter

10it [00:00, 5694.14it/s]

[14.33333333 15.33333333]
2.3333333333333335 3.6666666666666665
[11.66666667 16.66666667]
3.6666666666666665 4.166666666666667
[ 6.2 19.4]
6.4 5.8
[21.44444444 11.77777778]
-1.2222222222222223 1.4444444444444444
None
[-6. -5.]
24.0 18.0
[19.66666667 20.66666667]
-1.6666666666666667 -0.3333333333333333
[-2.  3.]
11.0 14.0
[19. 24.]
0.5 -1.0
[16. 39.]
-4.0 -4.0


2

# Part 2

In [79]:
with open("input24.txt") as f:
    hails =[Hail.fromfile(row.rstrip()) for row in f]

In [129]:
import sympy as sy

In [145]:
t1,t2,t3 = sy.Symbol("t_1"),sy.Symbol("t_2"),sy.Symbol("t_3")
x0,y0,z0 = sy.symbols("x,y,z")
vx0,vy0,vz0 = sy.symbols("vx,vy,vz")

In [146]:
def eqsystem(x,hails):
    x0 = x[:3]
    v0 = x[3:6]
    ts = x[6:]
    return np.concatenate([hail.pos-x0+t*(hail.vel-v0) for hail,t in zip(hails,ts)])

In [161]:
list(eqsystem((x0,y0,z0,vx0,vy0,vz0,t1,t2,t3),hails[:3]))

[t_1*(64 - vx) - x + 232488932265751,
 t_1*(273 - vy) - y + 93844132799095,
 t_1*(119 - vz) - z + 203172424390144,
 t_2*(14 - vx) - x + 258285813391475,
 t_2*(-vy - 10) - y + 225317967801013,
 t_2*(-vz - 22) - z + 306162724914014,
 t_3*(-vx - 182) - x + 377519381672953,
 t_3*(-vy - 80) - y + 343737262245611,
 t_3*(-vz - 373) - z + 485395777725108]

In [152]:
sol = sy.solve(list(eqsystem((x0,y0,z0,vx0,vy0,vz0,t1,t2,t3),hails[:3])))[0]
sol

{t_1: 654071052858,
 t_2: 857208450422,
 t_3: 556101734365,
 vx: -20,
 vy: -274,
 vz: 31,
 x: 287430900705823,
 y: 451620998712421,
 z: 260730677041648}

In [158]:
sol[x0]+sol[y0]+sol[z0]

999782576459892

In [156]:
sol.keys()

dict_keys([t_1, t_2, t_3, vx, vy, vz, x, y, z])